# Human-in-the-Loop (HITL) with Strands Agents

This tutorial demonstrates how to implement human-in-the-loop workflows with Strands Agents, enabling you to pause agent execution, request human input or approval, and resume based on that feedback.

| Feature | Description |
|---------|-------------|
| **Hook-Based Interrupts** | Intercept tool calls before execution using BeforeToolCallEvent |
| **Tool-Based Interrupts** | Raise interrupts directly from within tool definitions |
| **Session Persistence** | Remember user preferences across sessions with FileSessionManager |

## Architecture

Strands Agents provides a powerful interrupt system that allows you to pause execution at specific points, request human input, and resume with that response. There are two primary patterns for implementing interrupts:

**Pattern 1: Hook-Based Interrupts**
<div style="text-align:center">
    <img src="images/pattern-1.png" width="100%" />
</div>

**Pattern 2: Tool-Based Interrupts**
<div style="text-align:center">
    <img src="images/pattern-2.png" width="100%" />
</div>

## Setup and Prerequisites

### Prerequisites

- Python 3.10 or later
- AWS account with [Amazon Bedrock](https://aws.amazon.com/bedrock/) model access configured
- Basic understanding of Python programming
- Familiarity with Strands Agents basics [(see Quickstart Guide)](https://strandsagents.com/latest/documentation/docs/user-guide/quickstart/)

Let's now install the required dependencies:

### Importing dependency packages

In [ ]:
# Install required packages
!pip install -r requirements.txt --quiet

In [ ]:
# Standard library imports
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow")
import logging
import tempfile
from typing import Any

# Strands imports
from strands import Agent, tool
from strands.hooks import BeforeToolCallEvent, HookProvider, HookRegistry
from strands.types.tools import ToolContext
from strands.session import FileSessionManager

## Hook-Based Approval Workflow

Hooks allow you to intercept tool calls before they execute, making them perfect for approval workflows where you want human confirmation before performing sensitive operations. In this example, we'll create an agent that can delete files but requires human approval before doing so.

<div style="text-align:center">
    <img src="images/pattern-1.png" width="100%" />
</div>

### Configure logging

In [ ]:
# Configure logging for Strands
logging.getLogger("strands").setLevel(logging.INFO)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

### Define tools

We'll create two tools: one sensitive tool that needs approval (delete) and one safe tool (inspect).

In [ ]:
# Define tools: one sensitive (delete) and one safe (inspect)
@tool
def delete_files(paths: list[str]) -> str:
    """Delete files at the specified paths."""
    # In a real implementation, this would actually delete files
    return f"Successfully deleted files: {paths}"

@tool
def inspect_files(paths: list[str]) -> dict[str, Any]:
    """Inspect files and return their metadata."""
    # Dummy implementation for demo
    return {path: {"size": "1KB", "age": "10 days"} for path in paths}

### Create approval hook

The approval hook intercepts delete operations and requests human confirmation before proceeding.

In [ ]:
# Create an approval hook that intercepts delete operations
class ApprovalHook(HookProvider):
    def __init__(self, app_name: str) -> None:
        self.app_name = app_name

    def register_hooks(self, registry: HookRegistry, **kwargs: Any) -> None:
        # Register our approval callback for BeforeToolCallEvent
        registry.add_callback(BeforeToolCallEvent, self.approve)

    def approve(self, event: BeforeToolCallEvent) -> None:
        # Only intercept delete_files calls
        if event.tool_use["name"] != "delete_files":
            return

        # Raise an interrupt to request human approval
        # The interrupt() call pauses execution and returns control to the user
        approval = event.interrupt(
            f"{self.app_name}-approval", 
            reason={"paths": event.tool_use["input"]["paths"]}
        )
        
        # When execution resumes, 'approval' contains the user's response
        if approval.lower() != "y":
            # Cancel the tool execution with a message
            event.cancel_tool = "User denied permission to delete files"

### Key components explained

- **`HookProvider`**: Base class for creating hooks
- **`BeforeToolCallEvent`**: Event fired before a tool executes
- **`event.interrupt(name, reason)`**: Raises an interrupt with a unique identifier and optional context
- **`event.cancel_tool`**: Set to a message to cancel the tool execution

### Create the agent

In [ ]:
# Create the agent with our approval hook
agent = Agent(
    model="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    hooks=[ApprovalHook("filemanager")],
    system_prompt="You are a file management assistant. You can inspect and delete files. Always inspect files before deleting them.",
    tools=[delete_files, inspect_files],
    callback_handler=None,  # Disable streaming for cleaner output
)

### Run the agent with interrupt handling

**Note:** The cell below will prompt you for input.

In [ ]:
# Run the agent with interrupt handling
paths = ["old_report.txt", "temp_data.csv", "backup_2023.zip"]
result = agent(f"Please delete these old files: {paths}")

# Handle the interrupt loop
while True:
    # Check if the agent stopped due to an interrupt
    if result.stop_reason != "interrupt":
        break

    # Process each interrupt
    responses = []
    for interrupt in result.interrupts:
        if interrupt.name == "filemanager-approval":
            # Request human input
            print(f"\n APPROVAL REQUIRED")
            print(f"   Files to delete: {interrupt.reason['paths']}")
            user_input = input("   Do you approve? (y/N): ")
            
            # Build the response
            responses.append({
                "interruptResponse": {
                    "interruptId": interrupt.id,
                    "response": user_input
                }
            })

    # Resume the agent with the responses
    result = agent(responses)

# Print the final result
print(f"\n FINAL RESULT:")
print(f"   Stop Reason: {result.stop_reason}")
print(f"   Message: {result.message['content'] if result.message else 'No message'}")

## Tool-Based Interrupts

You can also raise interrupts directly from within your tool definitions. This is useful when the decision to pause depends on logic inside the tool itself. In this example, we'll create a tool that requests approval based on the number of files being deleted.

<div style="text-align:center">
    <img src="images/pattern-2.png" width="100%" />
</div>

### Define smart delete tool

This tool only requests approval when deleting multiple files (above a threshold).

In [ ]:
# Create tool that asks for approval based on number of files being deleted
class SmartDeleteTool:
    def __init__(self, app_name: str, approval_threshold: int = 3) -> None:
        self.app_name = app_name
        self.approval_threshold = approval_threshold

    @tool(context=True)
    def smart_delete(self, tool_context: ToolContext, paths: list[str]) -> str:
        """Delete files with smart approval, requests confirmation for bulk deletes."""
        
        # Only request approval if deleting many files
        if len(paths) >= self.approval_threshold:
            approval = tool_context.interrupt(
                f"{self.app_name}-bulk-approval",
                reason={
                    "paths": paths,
                    "count": len(paths),
                    "message": f"You are about to delete {len(paths)} files"
                }
            )
            
            if approval.lower() != "y":
                return f"Bulk delete cancelled by user. {len(paths)} files were NOT deleted."
        
        # Proceed with deletion (simulated)
        return f"Successfully deleted {len(paths)} files: {paths}"

### Create agent with smart delete tool

In [ ]:
# Create agent with the smart delete tool
smart_delete_tool = SmartDeleteTool("smartfile", approval_threshold=3)

agent_v2 = Agent(
    model="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="You are a file management assistant with smart deletion capabilities.",
    tools=[smart_delete_tool.smart_delete, inspect_files],
    callback_handler=None,
)

### Test with small number of files

When deleting fewer files than the threshold, no approval is needed.

In [ ]:
# Test with a small number of files (no approval needed)
print("=" * 50)
print("TEST 1: Deleting 2 files (below threshold)")
print("=" * 50)

result = agent_v2("Delete these files: file1.txt, file2.txt")
print(f"Result: {result.message['content'] if result.message else 'No message'}")

### Test with many files

When deleting more files than the threshold, approval is required.

**Note:** The cell below will prompt you for input.

In [ ]:
# Test with many files (approval required)
print("=" * 50)
print("TEST 2: Deleting 5 files (above threshold)")
print("=" * 50)

many_files = ["file1.txt", "file2.txt", "file3.txt", "file4.txt", "file5.txt"]
result = agent_v2(f"Delete these files: {many_files}")

while result.stop_reason == "interrupt":
    responses = []
    for interrupt in result.interrupts:
        if interrupt.name == "smartfile-bulk-approval":
            print(f"\n  BULK DELETE WARNING")
            print(f"   {interrupt.reason['message']}")
            print(f"   Files: {interrupt.reason['paths']}")
            user_input = input("   Proceed with bulk delete? (y/N): ")
            
            responses.append({
                "interruptResponse": {
                    "interruptId": interrupt.id,
                    "response": user_input
                }
            })
    
    result = agent_v2(responses)

print(f"\nResult: {result.message['content'] if result.message else 'No message'}")

## Session Management for Persistent Preferences

For production applications, you may want to persist interrupt state across sessions and remember user preferences (e.g., "always approve" or "trust mode"). Strands provides session management to handle these scenarios.

### Create persistent approval hook

This hook remembers user trust preferences across sessions.

In [ ]:
# Create hook that can persist approval status if user chooses
class PersistentApprovalHook(HookProvider):
    """Hook that remembers user's trust preferences across sessions."""
    
    def __init__(self, app_name: str) -> None:
        self.app_name = app_name

    def register_hooks(self, registry: HookRegistry, **kwargs: Any) -> None:
        registry.add_callback(BeforeToolCallEvent, self.approve_with_memory)

    def approve_with_memory(self, event: BeforeToolCallEvent) -> None:
        if event.tool_use["name"] != "delete_files":
            return

        # Check if user has already granted trust
        trust_key = f"{self.app_name}-trust"
        if event.agent.state.get(trust_key) == "t":
            print("   [Using saved trust preference: auto-approving]")
            return

        # Request approval with trust option
        approval = event.interrupt(
            f"{self.app_name}-approval",
            reason={
                "paths": event.tool_use["input"]["paths"],
                "options": "t=trust always, y=yes once, n=no"
            }
        )
        
        approval_lower = approval.lower()
        
        if approval_lower == "t":
            # Save trust preference to agent state (persisted by session manager)
            event.agent.state.set(trust_key, "t")
            print("   [Trust preference saved for future requests]")
        elif approval_lower != "y":
            event.cancel_tool = "User denied permission"

### Create agent with session management

In [ ]:
# Create agent with session management
# Note: In production, use a persistent storage_dir
# Create a temporary directory for session storage
session_dir = tempfile.mkdtemp()

agent_persistent = Agent(
    model="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    hooks=[PersistentApprovalHook("persistent-app")],
    session_manager=FileSessionManager(
        session_id="demo-session",
        storage_dir=session_dir
    ),
    system_prompt="You are a file management assistant.",
    tools=[delete_files, inspect_files],
    callback_handler=None,
)

print(f"Session storage: {session_dir}")

### Helper function for interrupt handling

In [ ]:
# Helper function to run agent with interrupt handling
def run_with_interrupts(agent, prompt):
    result = agent(prompt)
    
    while result.stop_reason == "interrupt":
        responses = []
        for interrupt in result.interrupts:
            print(f"\n {interrupt.name.upper()}")
            if "options" in interrupt.reason:
                print(f"   Options: {interrupt.reason['options']}")
            if "paths" in interrupt.reason:
                print(f"   Files: {interrupt.reason['paths']}")
            
            user_input = input("   Your choice: ")
            responses.append({
                "interruptResponse": {
                    "interruptId": interrupt.id,
                    "response": user_input
                }
            })
        
        result = agent(responses)
    
    return result

### First delete request

User can choose to trust the agent for future requests.

**Note:** The cell below will prompt you for input.

In [ ]:
# First request - user can choose to trust
print("=" * 50)
print("FIRST DELETE REQUEST")
print("=" * 50)

result = run_with_interrupts(agent_persistent, "Delete file1.txt")
print(f"\nResult: {result.message['content'] if result.message else 'No message'}")

### Second delete request

If you chose 't' (trust) in the previous cell, this will auto-approve.

**Note:** The cell below will prompt you for input if trust was not granted.

In [ ]:
# Second request - if you chose 't' (trust) in the previous cell, this will auto-approve
print("=" * 50)
print("SECOND DELETE REQUEST")
print("=" * 50)

result = run_with_interrupts(agent_persistent, "Delete file2.txt and file3.txt")
print(f"\nResult: {result.message['content'] if result.message else 'No message'}")

### Rerunning this section

If you would like to make different decisions in the cells above, run the cleanup cell below, then restart at the "Create agent with session management" cell.

## Clean up

Clean up the temporary session storage directory.

In [ ]:
# Cleanup session directory
import shutil
shutil.rmtree(session_dir, ignore_errors=True)
print("Session storage cleaned up.")

## Conclusion

In this notebook, you learned how to:

1. Implement hook-based approval workflows using BeforeToolCallEvent
2. Raise interrupts directly from within tools using tool_context.interrupt()
3. Handle interrupt loops and resume agent execution with user responses
4. Persist user preferences across sessions with FileSessionManager and agent state

### Key concepts summary

| Feature | Description |
|---------|-------------|
| **Hook Interrupts** | Intercept tool calls before execution using `BeforeToolCallEvent` |
| **Tool Interrupts** | Pause execution from within tools using `tool_context.interrupt()` |
| **Interrupt Response** | Resume execution by providing `interruptResponse` blocks |
| **Cancel Tool** | Prevent tool execution with `event.cancel_tool` |
| **Session Management** | Persist interrupt state and user preferences with `FileSessionManager` |
| **Agent State** | Store user preferences with `agent.state.set()` and `agent.state.get()` |

### Important notes

- Interrupt names must be unique within their scope (hook or tool)
- A single hook/tool can raise multiple interrupts sequentially, not simultaneously
- All concurrently running tools can raise interrupts independently

For more details, check out the [Strands Interrupts Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/interrupts/).

### Congratulations! 🎉

You've learned how to build human-in-the-loop workflows with Strands Agents. You can now:

- ✅ Create approval workflows using hooks
- ✅ Raise interrupts from within tools
- ✅ Handle multiple interrupts in a single execution
- ✅ Persist user preferences with session management

### Next Steps

- Explore the [Strands Agents documentation](https://strandsagents.com) for more advanced features
- Try combining interrupts with other Strands features like multi-agent systems
- Build your own approval workflows for your specific use cases

Happy building! 🚀